In [ ]:
import random
import math
from copy import deepcopy

# -------------------------------------------------
# Simple Tic-Tac-Toe board (3x3)
# 0 = empty, 1 = X (first player), -1 = O (second player)
# -------------------------------------------------
class Board:
    def __init__(self, board=None):
        if board is None:
            self.cells = [[0]*3 for _ in range(3)]
        else:
            self.cells = deepcopy(board.cells)
        self.current_player = 1                     # 1 starts

    def legal_moves(self):
        moves = []
        for r in range(3):
            for c in range(3):
                if self.cells[r][c] == 0:
                    moves.append((r, c))
        return moves

    def make_move(self, row, col):
        if self.cells[row][col] != 0:
            raise ValueError("Illegal move")
        new_board = Board(self)
        new_board.cells[row][col] = self.current_player
        new_board.current_player = -self.current_player
        return new_board

    def is_terminal(self):
        return self.winner() != 0 or len(self.legal_moves()) == 0

    def winner(self):
        # rows, cols, diagonals
        for i in range(3):
            if abs(sum(self.cells[i])) == 3:      return self.cells[i][0]
            if abs(sum(self.cells[r][i] for r in range(3))) == 3: return self.cells[0][i]
        if abs(self.cells[0][0] + self.cells[1][1] + self.cells[2][2]) == 3:
            return self.cells[0][0]
        if abs(self.cells[0][2] + self.cells[1][1] + self.cells[2][0]) == 3:
            return self.cells[0][2]
        return 0                                   # 0 = draw or ongoing

    def reward(self, root_player):
        w = self.winner()
        if w == root_player:   return 1
        if w == -root_player:  return -1
        return 0                                           # draw

    def print(self):
        symbols = {0: '.', 1: 'X', -1: 'O'}
        for row in self.cells:
            print(' '.join(symbols[c] for c in row))
        print()


# -------------------------------------------------
# MCTS Node
# -------------------------------------------------
class MCTSNode:
    def __init__(self, board, parent=None, move=None):
        self.board = board
        self.parent = parent
        self.move = move                     # (row, col) that led to this node
        self.children = {}                   # move → MCTSNode
        self.N = 0                           # visits
        self.W = 0.0                         # total reward
        self.untried_moves = board.legal_moves()

    def is_fully_expanded(self):
        return len(self.untried_moves) == 0

    def best_child(self, c_param=1.414):          # √2 ≈ 1.414
        if not self.children:
            return None
        log_n = math.log(self.N)
        return max(self.children.values(),
                   key=lambda child: (child.W / child.N if child.N > 0 else 0) +
                                    c_param * math.sqrt(log_n / child.N))

    def expand(self):
        move = self.untried_moves.pop()
        next_board = self.board.make_move(*move)
        child = MCTSNode(next_board, parent=self, move=move)
        self.children[move] = child
        return child

    def backpropagate(self, result):
        node = self
        while node is not None:
            node.N += 1
            node.W += result
            node = node.parent


# -------------------------------------------------
# Main MCTS function
# -------------------------------------------------
def mcts(root_board, iterations=8000, c_param=1.414):
    root = MCTSNode(root_board)

    for _ in range(iterations):
        node = root

        # 1. Selection
        while node.untried_moves == [] and node.children:
            node = node.best_child(c_param)

        # 2. Expansion
        if node.untried_moves and not node.board.is_terminal():
            node = node.expand()

        # 3. Simulation (random playout)
        board = node.board
        while not board.is_terminal():
            move = random.choice(board.legal_moves())
            board = board.make_move(*move)

        result = board.reward(root_board.current_player)   # from root player's view

        # 4. Backpropagation
        node.backpropagate(result)

    # Return most visited move
    best_move = max(root.children.items(),
                    key=lambda item: item[1].N)[0]
    return best_move, root


# -------------------------------------------------
# Play against the AI (you are X)
# -------------------------------------------------
def play_game():
    board = Board()
    print("You are X, AI is O. MCTS gets 8000 iterations per move.")
    board.print()

    while not board.is_terminal():
        if board.current_player == 1:                 # human turn
            while True:
                try:
                    r = int(input("Row (0-2): "))
                    c = int(input("Col (0-2): "))
                    board = board.make_move(r, c)
                    break
                except:
                    print("Invalid move, try again.")
        else:                                         # AI turn
            print("AI thinking...")
            move, root = mcts(board, iterations=10000)
            print(f"AI plays {move}")
            board = board.make_move(*move)

        board.print()

    w = board.winner()
    if w == 1:   print("X wins!")
    elif w == -1: print("O wins!")
    else:        print("Draw!")

if __name__ == "__main__":
    play_game()

You are X, AI is O. MCTS gets 8000 iterations per move.
. . .
. . .
. . .

Row (0-2): X
Invalid move, try again.
Row (0-2): 0
Col (0-2): 1
. X .
. . .
. . .

AI thinking...
AI plays (2, 2)
. X .
. . .
. . O

Row (0-2): 1
Col (0-2): 2
. X .
. . X
. . O

AI thinking...
AI plays (0, 0)
O X .
. . X
. . O

Row (0-2): 2
Col (0-2): 0
O X .
. . X
X . O

AI thinking...
AI plays (1, 1)
O X .
. O X
X . O

O wins!
